# Retrieving the Nth best redrock fit

This tutorial describes how to look up the Nth best redrock fit coefficients. 

See https://desi.lbl.gov/trac/wiki/Computing/JupyterAtNERSC to configure Jupyter at NERSC to be able to run this
tutorial using jupyter.nersc.gov and the fuji data in `/global/cfs/cdirs/desi/spectro/redux/fuji/tiles/cumulative/80618/20210328/`. This notebook runs on kernel 22.2. 

The bottom of this tutorial lists the code versions used while testing this.

Raffaella Capasso<br/>
July 2022

## Basic imports

In [ ]:
import os
from astropy.table import Table
import numpy as np
import desispec.io

import matplotlib.pyplot as plt
import redrock.templates
import h5py
import sys

Add the tutorials directory to the python search path so that we can import desitutorials

In [ ]:
sys.path.append(os.path.abspath(os.getcwd()+'/../py/'))

import desitutorials

Move into the directory that contains the reference run data and set some environment variables so that desispec can find the files.  If you are running locally (*e.g.* on your laptop instead of at jupyter.nersc.gov, replace these with wherever you installed the reference run data.

In [ ]:
os.chdir('/global/cfs/cdirs/desi/spectro/redux/fuji/tiles/cumulative/80618/20210328/')

Import redrock templates

In [ ]:
templates = dict()
for filename in redrock.templates.find_templates():
    # print('test')
    t = redrock.templates.Template(filename) #check if you can add a "verbose=False"
    templates[(t.template_type, t.sub_type)] = t


Read in the spectra and the matching zbest file. Also read the fibermap and trim to a unique set that row-matches the zbest file.

In [ ]:
zbest = Table.read('redrock-0-80618-thru20210328.fits', hdu=1).filled('')
spectra = desispec.io.read_spectra('coadd-0-80618-thru20210328.fits')

fm = Table.read('redrock-0-80618-thru20210328.fits', hdu='FIBERMAP')
ii=sorted(np.unique(fm['TARGETID'], return_index=True)[1])
assert np.all(fm['TARGETID'] == zbest['TARGETID'])
fm = fm[ii]

In [ ]:
goodELGs = (zbest['SPECTYPE'] == 'GALAXY')  
goodELGs &= (zbest['ZWARN'] == 0)  &  (zbest['DELTACHI2']>5)
goodELGs &= (fm['OBJTYPE'] == 'TGT')

In [ ]:
i = np.where(goodELGs)[0][0]
z = zbest['Z'][i]
targetid = zbest['TARGETID'][i]
spectype = zbest['SPECTYPE'][i].strip()
subtype = zbest['SUBTYPE'][i].strip()
fulltype = (spectype, subtype)
ncoeff = templates[fulltype].flux.shape[0]
coeff = zbest['COEFF'][i][0:ncoeff]

tflux = templates[fulltype].flux.T.dot(coeff)
twave = templates[fulltype].wave * (1+z)



In [ ]:
print(z,targetid)

In [ ]:
ispec = np.where(spectra.fibermap['TARGETID'] == targetid)[0][0]

plt.figure(figsize=(8,5))
plt.subplot(211)
maxflux = 0.0
for band in spectra.bands:
    plt.plot(spectra.wave[band], spectra.flux[band][ispec], 'k-', alpha=0.5)
    maxflux = max(maxflux, np.max(spectra.flux[band][ispec]))

plt.plot(twave, tflux, 'r-')
plt.xlim(3500, 10000)
# ylim(-1, 0.9*maxflux)
plt.ylim(-1, 8)

#- zoom in on [OII]
plt.subplot(212)
for band in spectra.bands:
    plt.plot(spectra.wave[band], spectra.flux[band][ispec], 'k-', alpha=0.5)

plt.plot(twave, tflux, 'r-')
plt.xlim(3727*(1+z)-100, 3727*(1+z)+100)
plt.ylim(-1,8)

## Look at the best 2nd fit

In [ ]:
filename = 'rrdetails-0-80618-thru20210328.h5'

data = h5py.File(filename, mode='r')
zfit = Table(np.array(data['zfit'][str(targetid)]['zfit']))
zfit['znum', 'z', 'spectype', 'zwarn', 'chi2', 'deltachi2']

In [ ]:
coeff_best=zfit['coeff'][0]
coeff_2nd=zfit['coeff'][1]
z_best=zfit['z'][0]
z_2nd=zfit['z'][1]

targetids = np.array(data['targetids'])
z_table = np.array(data['zscan']['GALAXY']['redshifts']) #scanned z
zchi2 = np.array(data['zscan']['GALAXY']['zchi2']) #500 targets per z
chi2_penalty = np.array(data['zscan']['GALAXY']['penalty'])

tflux_best = templates[fulltype].flux.T.dot(coeff_best)
twave_best = templates[fulltype].wave * (1+z_best)

tflux_2nd = templates[fulltype].flux.T.dot(coeff_2nd)
twave_2nd = templates[fulltype].wave * (1+z_2nd)

In [ ]:
plt.figure(figsize=(8,5))

ymin=-1.
ymax=8.

plt.subplot(211)

for band in spectra.bands:
    plt.plot(spectra.wave[band], spectra.flux[band][ispec], 'k-', alpha=0.5)

# plot(wave, flux, 'k-', alpha=0.5)
plt.plot(twave, tflux, 'r-')
plt.xlim(3500, 10000)
plt.ylim(ymin, ymax)


#- zoom in on [OII]
plt.subplot(212)
for band in spectra.bands:
    plt.plot(spectra.wave[band], spectra.flux[band][ispec], 'k-', alpha=0.5)

# plot(wave, flux, 'k-', alpha=0.5)
plt.plot(twave_best, tflux_best, 'r-')
plt.plot(twave_2nd, tflux_2nd, 'b-')
plt.xlim(3727*(1+z)-100, 3727*(1+z)+100)
plt.ylim(ymin, ymax)

## Code versions

In [ ]:
print('Code versions used for this tutorial:')
desitutorials.print_code_versions()